In [0]:
import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)

In [0]:
# download TA-Lib 
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz 

!ls
!tar xvzf ta-lib-0.4.0-src.tar.gz
!ls

import os
os.chdir('ta-lib') # Can't use !cd in co-lab
!./configure --prefix=/usr
!make install
os.chdir('../')

In [0]:
!pip install --target=$nb_path yfinance
!pip install --target=$nb_path TA-Lib
!pip install --target=$nb_path mplfinance

#!pip install -q yfinance
#!pip install -q TA-Lib

In [0]:
# Import the libraries
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import talib as ta
import plotly.offline as py
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import mplfinance

plt.style.use('fivethirtyeight')

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [0]:
# valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
# valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo

def get_historical_data(ticker, period, interval):
    data = yf.download(
        tickers = ticker,
        period = period,
        interval = interval,
        group_by = 'ticker',
        auto_adjust = True,
        prepost = True)
    return data

In [0]:
data = get_historical_data(["MSFT", "SPY"], "2y", "1d"); data.head()

[*********************100%***********************]  2 of 2 completed


MSFT                        ...         SPY                       
                 Open       High        Low  ...         Low       Close     Volume
Date                                         ...                                   
2018-05-29  95.106282  96.117225  94.513333  ...  257.404746  258.615997  115908600
2018-05-30  95.563155  96.476893  95.174337  ...  259.961831  262.067108   69678400
2018-05-31  96.515785  97.196224  95.854784  ...  259.808036  260.461731   93519900
2018-06-01  96.506058  98.041913  96.399130  ...  261.797957  263.018860   71258400
2018-06-04  98.430744  99.013978  98.032196  ...  263.653352  264.268585   45385200

[5 rows x 10 columns]

In [0]:
def transform_data(df):
    df = pd.DataFrame(df)
    df = df.resample('1D').ffill()
    df = df.drop_duplicates()
    df.index = df.set_index()
    return df

In [0]:
data = transform_data(data); data[data.duplicated()]

TypeError: ignored

In [0]:
# Change stock to DF since that is the input
def addFeatures(stock):
    data = stock
    data['daily_change'] = data.Close.pct_change()*100
    data['weekly_change'] = data['Close'].pct_change(periods=7)
    data['monthly_change'] = data['Close'].pct_change(periods=30)
    
    # Simple Moving Averages
    data['SMA_50'] = ta.SMA(data.Close, timeperiod=50)
    data['SMA_200'] = ta.SMA(data.Close, timeperiod=200)  

    # Average True Range
    data['ATR_14'] = ta.ATR(high = data.High, low = data.Low, close = data.Close, timeperiod=14)

    # Relative Strength Index
    data['RSI_14'] = ta.RSI(data.Close, timeperiod=14)  

    #MACD
    data['macd'], data['macdSignal'], data['macdHist'] = ta.MACD(data.Close, fastperiod=12, slowperiod=26, signalperiod=9)

    # Test
    data['SMA_test'] = np.where(data.SMA_50 > data.SMA_200, 1, 0)
    data['macd_test'] = np.where((data.macd > data.macdSignal), 1, 0)
    return data 

In [0]:
df = addFeatures(stock=data); df.head()

,Open,High,Low,Close,Volume,daily_change,weekly_change,monthly_change,SMA_50,SMA_200,ATR_14,RSI_14,macd,macdSignal,macdHist,SMA_test,macd_test
Date,,,,,,,,,,,,,,,,,
2018-05-29,95.106282,96.117225,94.513333,95.271538,28671000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2018-05-30,95.563155,96.476893,95.174337,96.185272,22158500,0.959084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2018-05-31,96.515785,97.196224,95.854784,96.078354,34140900,-0.111159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2018-06-01,96.506058,98.041913,96.399130,97.973869,28655600,1.972885,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2018-06-04,98.430744,99.013978,98.032196,98.829285,27281600,0.873106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [0]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x = data.index,
    y = data.Close,
    name= "Close"
))

fig.add_trace(go.Scatter(
    x = data.index,
    y = data.SMA_200,
    name = "SMA 200"
))

fig.add_trace(go.Scatter(
    x = data.index,
    y = data.SMA_50,
    name = "SMA 50"
))

fig.update_layout(
    title = "Microsoft",
    xaxis_title = "Date",
    yaxis_title = "Close"
)

fig.update_layout(height=900, width=1300)

fig.show()

In [0]:
fig = make_subplots(rows=3, cols=1, 
                    shared_xaxes=True, 
                    vertical_spacing=0.02,
                    subplot_titles=("Close price of S&P 500", "RSI 14 days", "MACD"))

fig.add_trace(go.Scatter(x=list(data.index), 
                         y=list(data.Close),
                         name = "Close"),
                          row=1, col=1)

fig.add_trace(go.Scatter(x=list(data.index), 
                         y=list(data.RSI_14),
                         name = "RSI 14"),
              row=2, col=1)

# RSI above 70
fig.add_shape(type="line",
            x0=min(data.index),
            y0=30,
            x1=max(data.index),
            y1=30,
            line=dict(color="LightSeaGreen",width=4,dash="dashdot"),
            row=2, col=1
    )

# RSI below 30
fig.add_shape(type="line",
            x0=min(data.index),
            y0=70,
            x1=max(data.index),
            y1=70,
            line=dict(color="LightSeaGreen",width=3,dash="dashdot"),
            row=2, col=1
    )

fig.add_trace(go.Scatter(x=list(data.index), 
                         y=list(data.macd),
                         name = "MACD"),
              row=3, col=1)

fig.add_trace(go.Scatter(x=list(data.index), 
                         y=list(data.macdSignal),
                         name = "macdSignal"),
              row=3, col=1)

fig.update_layout(height=900, width=1300)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1d",
                     step="day",
                     stepmode="backward"),
                dict(count=7,
                     label="1w",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=3,
                     label="3m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(count=3,
                     label="3y",
                     step="year",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=False
        ),
        type="date"
    )
)

#fig.update_layout(legend_orientation="h", xaxis3_rangeslider_visible=True)
fig.show()

In [0]:
  # Kommentarer från Week5
  # Se till att Github versionshantering är på plats
  # Kolla upp varför jag får 16 observationer per heldag. Hur många borde jag få?
  # Se till att index skrivs ut som ett datetime och inte date (dvs visa även timmar)
  # Kopiera över koden till JupyterLabs och kör därifrån
  # Se över hur man får in fler aktier än bara en
  # Kör en groupby per aktie
  # Gör en funktion för visualisering med flera inputs
  # Bonus - läs på om Flask Python